In [1]:
import torch
import torch_geometric
from torch_geometric.data import Data
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import optuna
import training
import models
import metrics
from models import SpatioTemporalAutoencoder, FixedGraphTransformerNormalizingFlow
from parameters import STAEParameters, TrainingParameters
from datautils import get_morning_data, normalize_data, generate_edges, generate_relational_edges, load_best_parameters, get_full_data, label_anomalies
from training import save_model, load_model, train_f_gtnf, compute_anomaly_threshold_rstae, test_model, threshold_anomalies, fill_result_df, test_rstae
from metrics import calculate_accuracy, crash_detection_delay, calculate_tp_fp, find_thresholds, find_delays, find_percent, discrete_fp_delays, generate_anomaly_labels, calculate_auc, discrete_fp_delays_loc_nll
import torch.nn as nn
from tqdm import tqdm

import random
import os
seed=42
torch_geometric.seed_everything(seed)


def sequence_stae(data, timesteps, hide_anomalies=False):
    sequence = []
    relational_edges, relations = generate_relational_edges(milemarkers=list(range(49)), timesteps=timesteps)
    static_edges = generate_edges(milemarkers=list(range(49)))
    days = data['day']
    anomalies = data['anomaly']
    data_vals = data[['occ', 'speed', 'volume']]
    unix = data['unix_time']
    unique_times = np.unique(data['unix_time'])
    kept_indices = []

    for index, t in enumerate(tqdm(unique_times[timesteps:])): # skip first 'timesteps'
        data_t = []
        backward_index = range(index-1, index-timesteps-1, -1)
        backward_times = [unique_times[i] for i in backward_index]
        curr_day = np.unique(data[data['unix_time']==backward_times[-1]]['day'])[0]
        contains_anomaly = np.any([np.unique(data[data['unix_time']==i]['anomaly'])[0] for i in backward_times])
        is_curr_day = np.all([np.unique(data[data['unix_time']==i]['day'])[0]==curr_day for i in backward_times])

        if (hide_anomalies and contains_anomaly) or not is_curr_day:
            continue
        
        kept_indices.append(index+timesteps)

        for i in backward_times:
            data_t.append(Data(x=torch.tensor(data[data['unix_time']==i][['occ', 'speed', 'volume']].to_numpy(), dtype=torch.float32), edge_index=static_edges)) # assumes time indices come sequentially, with full data it may not

        curr_graph = data_t[0]
        sequence.append([data_t[::-1], curr_graph])

    return sequence, kept_indices

hide_anomalies = False
optimal_params = load_best_parameters('f_gtnf',hide_anomalies=hide_anomalies)
print(optimal_params)
training_params = TrainingParameters(
    learning_rate=optimal_params['learning_rate'],
    batch_size=1,
    timesteps=optimal_params['timesteps'],
    n_epochs=optimal_params['epochs'],
)

{'dropout_rate': 0.284365661252853, 'epochs': 8, 'flow_layers': 2, 'hidden_dim': 256, 'learning_rate': 1.4186618448297526e-06, 'n_hidden_flow': 3, 'num_gcn_layers': 1, 'num_transformer_layers': 1, 'output_dim': 128, 'timesteps': 2}


In [2]:
data, test_data, _ = get_full_data()
data = normalize_data(data)
data = label_anomalies(data)
length = len(data.day.unique())
train_length = int(length * 0.8)
val_length = length - train_length
train_days = data.day.unique()[:train_length]
val_days = data.day.unique()[train_length:]

# Use .isin() to filter the DataFrame based on the days
train_data = data[data.day.isin(train_days)]
val_data = data[data.day.isin(val_days)]
train_sequence, kept_train_indices = sequence_stae(train_data, training_params.timesteps, hide_anomalies=hide_anomalies)
val_sequence, kept_val_indices = sequence_stae(val_data, training_params.timesteps, hide_anomalies=hide_anomalies)


100%|██████████| 2878/2878 [00:06<00:00, 479.31it/s]


In [3]:


num_features = 3
num_nodes = 196

model_params = {
    'input_features': num_features,
    'num_sensors': num_nodes,
    'hidden_dim': optimal_params['hidden_dim'],
    'num_heads': 1,
    'num_transformer_layers': optimal_params['num_transformer_layers'],
    'output_dim': optimal_params['output_dim'],
    'num_gcn_layers': optimal_params['num_gcn_layers'],
    'flow_layers': optimal_params['flow_layers'],
    'n_hidden_flow':optimal_params['n_hidden_flow'],
    'dropout_rate':optimal_params['dropout_rate'],

    
}
using_pretrained = True

In [4]:


if not using_pretrained:
    model, losses= train_f_gtnf(model_params, training_params, train_sequence,verbose=True)
    save_model(model, f'f_gtnf_{hide_anomalies}_{seed}_both' )
    plt.plot(losses)

else:
    # model=FixedGraphTransformerNormalizingFlow(**model_params)
    model = load_model(FixedGraphTransformerNormalizingFlow,model_params,f'f_gtnf_{hide_anomalies}_{seed}')
    

    

In [7]:

thresh = training.compute_threshold_f_gtnf(val_sequence,model , 'max')

In [8]:
_, df_test_data, _ = get_full_data()
test_data = normalize_data(df_test_data)
test_data = label_anomalies(test_data)
test_data, kept_test_indices = sequence_stae(test_data, training_params.timesteps, hide_anomalies=False)

100%|██████████| 4798/4798 [00:13<00:00, 359.66it/s]


In [9]:
test_errors=training.test_f_gtnf(test_data, model, verbose=True)


100%|██████████| 4793/4793 [00:25<00:00, 185.50it/s]


In [ ]:
# Compute true anomaly labels
anomaly_labels = generate_anomaly_labels(df_test_data, kept_test_indices)

# Whether a crash was reported at each time
# crash_reported = df_test_data['crash_record'].to_numpy()[0::196][kept_test_indices]

/media/vanderbilt/home/zulqarnain/freeway-anomaly-detection/code/metrics.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.loc[(test_data['unix_time'] - human_label_time <= 1800) & (test_data['unix_time'] - human_label_time >= 0), 'anomaly'] = 1
/media/vanderbilt/home/zulqarnain/freeway-anomaly-detection/code/metrics.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.fillna(0, inplace=True)


In [11]:
converted_dates_utc = pd.to_datetime(df_test_data.unix_time, unit='s', utc=True)

# Step 2: Convert to US/Central timezone
converted_dates_central = converted_dates_utc.dt.tz_convert('US/Central')
converted_dates_naive = converted_dates_central.dt.tz_localize(None)
df_test_data['Time']=converted_dates_naive

import glob
pattern=sorted(glob.glob('../data/event_data/2023-10-*.csv'))
events=pd.DataFrame()
for i in pattern:
    event=pd.read_csv(i,sep=';')
    events=pd.concat([events,event],axis=0)
events.reset_index(drop=True,inplace=True)
# events['timestamp']=pd.to_datetime(events['event_update_time'],utc=False)
events['Time'] = pd.to_datetime(events['event_update_time'], utc=False)

# To ensure the timestamps are naive (no timezone)
events['Time'] = events['Time'].dt.tz_localize(None).dt.floor('1s').dt.ceil('30s')
events=events[(events['event_update_type'] == 'new') & (events['classification']=='incident')]
events.drop_duplicates(subset=['event_id'],inplace=True)
events=events.sort_values(by='Time').reset_index(drop=True)


array1=np.array(events.milemarker.sort_values().unique())
array2=np.array(df_test_data.milemarker.unique())
def find_closest_or_exact(target, array):
    # Find the indices where the target would fit
    idx = np.searchsorted(array, target)
    
    # Check for exact match
    if idx < len(array) and array[idx] == target:
        return array[idx]  # Exact match
    
    # Find the closest lower value (if it exists)
    lower = array[idx - 1] if idx > 0 else None
    
    # Find the closest higher value (if it exists)
    higher = array[idx] if idx < len(array) else None
    
    return [lower, higher]

# Mapping from array2 to closest values in array1
events['mapped_milemarkers'] = events['milemarker'].apply(lambda x: find_closest_or_exact(x, array2))
events=events.explode('mapped_milemarkers')
# Filter events based on Time in df_test_data
test_events = events[events['Time'].isin(df_test_data['Time'])]

# Ensure 'Time' in test_events is properly converted to pandas.Timestamp
test_events['Time'] = pd.to_datetime(test_events['Time'])

df_test_data['combined'] = df_test_data['Time'].astype(str) + '_' + df_test_data['milemarker'].astype(str)
test_events['combined'] = test_events['Time'].astype(str) + '_' + test_events['mapped_milemarkers'].astype(str)

df_test_data['exists'] = df_test_data['combined'].isin(test_events['combined']).astype(int)
df_test_data['crash_record']=((df_test_data['exists'] == 1) & (df_test_data['crash_record'] == 1)
).astype(int)

crash_reported=df_test_data['crash_record'].to_numpy().reshape(-1,196)[kept_test_indices]

results = discrete_fp_delays_loc_nll(thresh, test_errors, anomaly_labels, crash_reported)
path_results = 'saved_results_loc/trace/'
if not os.path.exists(path_results):
    os.makedirs(path_results)
results.to_csv(f'{path_results}results_{hide_anomalies}_{seed}.csv', index=False)



/tmp/ipykernel_15492/422466712.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_data['Time']=converted_dates_naive
/tmp/ipykernel_15492/422466712.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_events['Time'] = pd.to_datetime(test_events['Time'])
/tmp/ipykernel_15492/422466712.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[[1.0, -884.5032348632812], [1.0, -883.2387189880371], [1.0, -882.9436080200195], [1.0, -882.7465438964844], [1.0, -882.5764579833984], [1.0, -882.4196810913086], [1.0, -882.3031484008789], [1.0, -882.2158706176758], [1.0, -882.1166178710937], [1.0, -882.028272692871], [1.0, -881.9499538574219], [1.0, -881.8557447021484], [1.0, -881.7830887207032], [1.0, -881.7017510864258], [1.0, -881.6373559326172], [1.0, -881.572773071289], [1.0, -881.5161744140624], [1.0, -881.4627543273925], [1.0, -881.4098683227539], [1.0, -881.3511175292969], [1.0, -881.2970073242187], [1.0, -881.2378922363281], [1.0, -881.1874123291016], [1.0, -881.1385081726074], [1.0, -881.0895310058594], [1.0, -881.0427774047852], [1.0, -880.9992277221679], [1.0, -880.958318737793], [1.0, -880.8953911132812], [1.0, -880.8445595581055], [1.0, -880.8030834350586], [1.0, -880.7550450317383], [1.0, -880.7115072265625], [1.0, -880.6751586853028], [1.0, -880.6195489013672], [1.0, -880.5810560302734], [1.0, -880.5404474365234], [1.

In [ ]:
import metrics
import importlib
importlib.reload(metrics)
path_results = 'saved_results/f_gtnf/'
results = metrics.discrete_fp_delays_loc_nll(thresh, test_errors, anomaly_labels, crash_reported)

# Check if the directory exists, and create it if it does not
if not os.path.exists(path_results):
    os.makedirs(path_results)

# Save the results to a CSV file
# results.to_csv(f'{path_results}results_{hide_anomalies}_{seed}.csv', index=False)

100%|██████████| 10000/10000 [05:40<00:00, 29.35it/s]


Found FPR of 0.006218905472636816 for 0.01
Found FPR of 0.026533996683250415 for 0.025
Found FPR of 0.04892205638474295 for 0.05
Found FPR of 0.10240464344941957 for 0.1
Found FPR of 0.20107794361525705 for 0.2
FPR 1% gives mean delay of -15.0 +/- 0.0 while missing 0.9166666666666666%.
FPR 2.5% gives mean delay of -6.5 +/- 8.436033823229176 while missing 0.75%.
FPR 5% gives mean delay of -4.428571428571429 +/- 8.591642830717083 while missing 0.41666666666666663%.
FPR 10% gives mean delay of -7.1875 +/- 6.986314747418699 while missing 0.33333333333333337%.
FPR 20% gives mean delay of -10.625 +/- 4.484347778663024 while missing 0.33333333333333337%.


In [13]:
calculate_auc(test_errors,anomaly_labels)

0.7124574069018513